In [1]:
!pip install gradio transformers torch accelerate requests pypdf2 python-pptx beautifulsoup4 google-generativeai langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 19.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=1febb20bac89acd47ca2de29d114284549712487f845bdbf7fc744bef7c333ce
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
# Educational AI Assistant - Device Fixed + Dynamic Interface
# Run this in Google Colab

# Install required packages
!pip install gradio transformers torch accelerate requests pypdf2 python-pptx beautifulsoup4 google-generativeai langdetect

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import google.generativeai as genai
import os
import requests
from PyPDF2 import PdfReader
from pptx import Presentation
from bs4 import BeautifulSoup
from langdetect import detect
import json
import re
from datetime import datetime
import time

# Configuration
class Config:
    granite_model = "ibm-granite/granite-3.3-2b-instruct"
    gemini_model = "gemini-2.5-flash-image-preview"
    device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize models with proper device handling
def initialize_models(gemini_api_key):
    """Initialize both Granite and Gemini models with proper device handling"""
    try:
        # Initialize Granite model with proper device handling
        print(f"Loading AI models on {Config.device}...")
        tokenizer = AutoTokenizer.from_pretrained(Config.granite_model, trust_remote_code=True)

        # Add padding token if it doesn't exist
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            Config.granite_model,
            torch_dtype=torch.float16 if Config.device == "cuda" else torch.float32,
            device_map="auto" if Config.device == "cuda" else None,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )

        # Explicitly move model to device if not using device_map
        if Config.device == "cpu":
            model = model.to(Config.device)

        # Initialize Gemini
        genai.configure(api_key=gemini_api_key)
        gemini_model = genai.GenerativeModel(Config.gemini_model)

        return tokenizer, model, gemini_model, f"Ready! Models loaded on {Config.device}. AI assistant is now active."
    except Exception as e:
        return None, None, None, f"Setup failed: {str(e)}"

# File processing functions
def extract_text_from_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def extract_text_from_pptx(file_path):
    try:
        prs = Presentation(file_path)
        text = ""
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text += shape.text + "\n"
        return text
    except Exception as e:
        return f"Error reading PPTX: {str(e)}"

def extract_text_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text()
        return ' '.join(text.split())
    except Exception as e:
        return f"Error reading URL: {str(e)}"

def process_file(file, url_input):
    if file is not None:
        file_path = file.name
        if file_path.endswith('.pdf'):
            return extract_text_from_pdf(file_path)
        elif file_path.endswith('.pptx'):
            return extract_text_from_pptx(file_path)
        elif file_path.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return "Please upload PDF, PPTX, or TXT files only."
    elif url_input.strip():
        return extract_text_from_url(url_input.strip())
    else:
        return ""

# AI response generation with proper device handling
def generate_granite_response(tokenizer, model, prompt, max_length=1000):
    try:
        # Ensure all tensors are on the same device
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
            padding=True
        )

        # Move inputs to the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,  # Use max_new_tokens instead of max_length
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3
            )

        # Decode only the new tokens
        new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True)

        return response.strip() if response.strip() else "I apologize, but I couldn't generate a proper response. Please try rephrasing your request."

    except Exception as e:
        return f"Error generating response: {str(e)}"

def generate_flowchart(gemini_model, content):
    try:
        prompt = f"""
        Create a clear, step-by-step flowchart for this content:

        {content[:2000]}

        Format as:
        START → Step 1 → Decision Point? → Step 2 → END

        Use simple language and clear connections.
        """

        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Flowchart generation failed: {str(e)}"

# Enhanced AI processing function with streaming simulation
def process_ai_request(tokenizer, model, gemini_model, content, request_type, additional_input="", language="English"):
    """Enhanced function to handle all AI requests with better prompts"""

    if not content and request_type not in ["study_plan", "task_management"]:
        return "Please upload a file, enter a URL, or provide content first."

    # Enhanced prompt templates
    prompts = {
        "explain": f"""
        You are an expert educator. Explain the following content in simple, clear bullet points:

        Content: {content[:3000]}

        Instructions:
        • Use simple, accessible language
        • Break down complex concepts into digestible parts
        • Provide practical examples where relevant
        • Include key takeaways
        • Make it beginner-friendly

        Format your response with clear bullet points using the • symbol.
        """,

        "questions": f"""
        You are a knowledgeable tutor. Based on the content provided, answer the student's question thoroughly:

        Content: {content[:2500]}

        Student Question: {additional_input}

        Provide a comprehensive answer that:
        • Directly addresses the question
        • Uses examples from the content when possible
        • Explains any technical terms
        • Offers additional context if helpful
        """,

        "study_plan": f"""
        Create a comprehensive {additional_input} study plan for the topic: {content}

        Structure your plan with:

        LEARNING OBJECTIVES:
        • Clear, measurable goals

        STUDY SCHEDULE:
        • Daily/weekly breakdown
        • Time allocations

        KEY RESOURCES:
        • Essential materials to study
        • Recommended readings

        PRACTICE ACTIVITIES:
        • Hands-on exercises
        • Real-world applications

        ASSESSMENT METHODS:
        • Self-evaluation techniques
        • Progress milestones

        Make it practical and actionable for the specified duration.
        """,

        "tutor": f"""
        Act as a specialized virtual tutor for: {content}
        Student's Learning Style: {additional_input}

        Provide a personalized tutoring approach that includes:

        TEACHING STRATEGY:
        • Methods suited to their learning style
        • Engagement techniques

        LEARNING PATH:
        • Step-by-step progression
        • Key milestones

        PRACTICE METHODS:
        • Exercises tailored to learning style
        • Interactive activities

        COMMON CHALLENGES:
        • Typical difficulties students face
        • Strategies to overcome them

        PROGRESS EVALUATION:
        • How to measure understanding
        • Self-assessment techniques
        """,

        "translate": f"""
        Translate and explain the following content in {language}:

        Content: {content[:2000]}

        Provide:
        1. Accurate translation of key concepts
        2. Cultural context where relevant
        3. Simple explanations in the target language
        4. Important terminology with definitions

        Make sure the explanation is clear and educational.
        """,

        "tasks": f"""
        Apply the 1-3-5-7 productivity rule to organize this task efficiently:

        Main Task: {content}

        Organize as follows:

        TOP PRIORITY (1 task):
        • [The single most critical/urgent item that must be completed]

        MEDIUM PRIORITIES (3 tasks):
        • [Important task 1]
        • [Important task 2]
        • [Important task 3]

        SMALL TASKS (5 tasks):
        • [Manageable task 1]
        • [Manageable task 2]
        • [Manageable task 3]
        • [Manageable task 4]
        • [Manageable task 5]

        QUICK TASKS (7 tasks):
        • [Quick win 1]
        • [Quick win 2]
        • [Quick win 3]
        • [Quick win 4]
        • [Quick win 5]
        • [Quick win 6]
        • [Quick win 7]

        Make each item actionable and specific.
        """,

        "summarize": f"""
        Analyze and create a comprehensive summary of this content:

        Content: {content[:3000]}

        EXECUTIVE SUMMARY:
        • [Brief overview in 2-3 sentences]

        KEY POINTS:
        • [Main idea 1]
        • [Main idea 2]
        • [Main idea 3]

        IMPORTANT DETAILS:
        • [Critical information that supports key points]

        CONCLUSIONS/TAKEAWAYS:
        • [What the reader should remember]

        ACTION ITEMS (if applicable):
        • [What should be done based on this content]
        """
    }

    if request_type == "flowchart":
        return generate_flowchart(gemini_model, content)

    prompt = prompts.get(request_type, prompts["explain"])
    return generate_granite_response(tokenizer, model, prompt, 1200)

# Enhanced dynamic interface with real-time features
def create_dynamic_interface():
    # Global variables for models
    global granite_tokenizer, granite_model, gemini_model, processing_state
    granite_tokenizer, granite_model, gemini_model = None, None, None
    processing_state = {"is_processing": False, "current_task": ""}

    def initialize_system(api_key):
        global granite_tokenizer, granite_model, gemini_model
        if not api_key.strip():
            return "Please enter your Gemini API key", gr.update(visible=False)

        granite_tokenizer, granite_model, gemini_model, status = initialize_models(api_key)
        models_ready = granite_model is not None
        return status, gr.update(visible=models_ready)

    def update_processing_status(is_processing, task=""):
        processing_state["is_processing"] = is_processing
        processing_state["current_task"] = task
        if is_processing:
            return f"Processing: {task}...", gr.update(interactive=False)
        else:
            return "Ready for new requests", gr.update(interactive=True)

    def handle_request(file, url, text_input, request_type, question, study_duration, learning_style, language, progress=gr.Progress()):
        if granite_model is None:
            return "Please set up your API key first", "Ready"

        # Update processing status
        task_names = {
            "explain": "Explaining content",
            "questions": "Answering question",
            "flowchart": "Creating flowchart",
            "study_plan": "Building study plan",
            "tutor": "Setting up virtual tutor",
            "translate": "Translating content",
            "tasks": "Organizing tasks",
            "summarize": "Summarizing content"
        }

        current_task = task_names.get(request_type, "Processing request")

        # Simulate progress updates
        progress(0.1, desc="Analyzing input...")
        time.sleep(0.5)

        # Get content from file, URL, or text input
        content = ""
        if file is not None or url.strip():
            progress(0.3, desc="Processing file/URL...")
            content = process_file(file, url)
            time.sleep(0.5)
        elif text_input.strip():
            content = text_input.strip()

        progress(0.5, desc="Preparing AI request...")

        # Handle different request types
        additional_input = ""
        if request_type == "questions":
            additional_input = question
        elif request_type == "study_plan":
            additional_input = study_duration
            content = text_input if text_input.strip() else "General study topic"
        elif request_type == "tutor":
            additional_input = learning_style
        elif request_type == "tasks":
            content = text_input if text_input.strip() else content

        progress(0.7, desc=f"{current_task}...")

        result = process_ai_request(
            granite_tokenizer, granite_model, gemini_model,
            content, request_type, additional_input, language
        )

        progress(1.0, desc="Complete!")

        return result, "Ready for new requests"

    def get_content_preview(file, url, text_input):
        """Dynamic content preview"""
        if file is not None:
            return f"File uploaded: {file.name} | Type: {file.name.split('.')[-1].upper()}"
        elif url.strip():
            return f"URL provided: {url[:50]}{'...' if len(url) > 50 else ''}"
        elif text_input.strip():
            preview = text_input[:100]
            return f"Text input: {preview}{'...' if len(text_input) > 100 else ''}"
        else:
            return "No content provided yet"

    def update_button_states(request_type):
        """Update button appearances based on selection"""
        button_configs = {}
        for btn_type in ["explain", "questions", "flowchart", "summarize", "study_plan", "tutor", "tasks", "translate"]:
            if btn_type == request_type:
                button_configs[btn_type] = gr.update(variant="primary")
            else:
                button_configs[btn_type] = gr.update(variant="secondary")
        return button_configs

    # Enhanced CSS with animations and better responsiveness
    custom_css = """
    .gradio-container {
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif !important;
        max-width: 1400px !important;
        margin: 0 auto !important;
    }
    .main-header {
        text-align: center;
        padding: 2rem 1rem;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 1rem;
        margin-bottom: 2rem;
        box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
    }
    .content-preview {
        background: #f8f9fa;
        border: 1px solid #dee2e6;
        border-radius: 0.5rem;
        padding: 0.75rem;
        margin: 0.5rem 0;
        font-size: 0.9rem;
        color: #6c757d;
    }
    .status-indicator {
        padding: 0.5rem 1rem;
        border-radius: 0.375rem;
        text-align: center;
        font-weight: 500;
        margin: 0.5rem 0;
        transition: all 0.3s ease;
    }
    .status-ready {
        background: #d1edff;
        color: #0969da;
        border: 1px solid #54aeff;
    }
    .status-processing {
        background: #fff8dc;
        color: #9a6700;
        border: 1px solid #d4ac0d;
    }
    .quick-action-btn {
        margin: 0.25rem;
        min-height: 2.5rem !important;
        border-radius: 0.5rem !important;
        font-weight: 500 !important;
        transition: all 0.2s ease !important;
    }
    .quick-action-btn:hover {
        transform: translateY(-1px);
        box-shadow: 0 2px 8px rgba(0,0,0,0.15);
    }
    .results-container {
        border: 1px solid #e1e5e9;
        border-radius: 0.75rem;
        background: #ffffff;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    @media (max-width: 768px) {
        .main-header h1 { font-size: 1.5rem; }
        .quick-action-btn { min-height: 3rem !important; font-size: 0.9rem; }
    }
    """

    with gr.Blocks(css=custom_css, title="Edututor-AI Learning Assistant") as app:

        # Header
        gr.HTML("""
        <div class="main-header">
            <h1>Edututor</h1>
            <p>Powered by IBM Granite and gemini's nano banana</p>
            <p>Upload → Ask → Learn | Simple, Fast, Smart</p>
        </div>
        """)

        # Setup section with better feedback
        with gr.Accordion("Setup (Click to expand)", open=False):
            gr.Markdown("### Get your free API key from [Google AI Studio](https://aistudio.google.com/app/apikey)")
            with gr.Row():
                api_key = gr.Textbox(
                    label="Gemini API Key",
                    type="password",
                    placeholder="Enter your API key here...",
                    scale=3
                )
                setup_btn = gr.Button("Initialize", variant="primary", scale=1)

            setup_status = gr.Textbox(label="Status", interactive=False, show_label=False)

        # Main interface (initially hidden until setup)
        main_interface = gr.Column(visible=False)

        with main_interface:
            # Dynamic status indicator
            status_display = gr.HTML(
                '<div class="status-indicator status-ready">Ready for new requests</div>',
                visible=True
            )

            # Content input section
            gr.Markdown("## Add Your Content")

            with gr.Row():
                with gr.Column(scale=2):
                    file_input = gr.File(
                        label="Upload File",
                        file_types=[".pdf", ".pptx", ".txt"],
                        height=100
                    )
                with gr.Column(scale=2):
                    url_input = gr.Textbox(
                        label="Or Paste URL",
                        placeholder="https://example.com/article",
                        lines=1
                    )

            text_input = gr.Textbox(
                label="Or Type/Paste Text Directly",
                placeholder="Enter your content, question, or topic here...",
                lines=3
            )

            # Dynamic content preview
            content_preview = gr.HTML(
                '<div class="content-preview">No content provided yet</div>'
            )

            # Action buttons with enhanced layout
            gr.Markdown("## What would you like to do?")

            with gr.Row():
                explain_btn = gr.Button("Explain Simply", variant="secondary", size="lg")
                questions_btn = gr.Button("Ask Questions", variant="secondary", size="lg")
                flowchart_btn = gr.Button("Create Flowchart", variant="secondary", size="lg")
                summarize_btn = gr.Button("Summarize", variant="secondary", size="lg")

            with gr.Row():
                study_btn = gr.Button("Study Plan", variant="secondary", size="lg")
                tutor_btn = gr.Button("Get Tutor", variant="secondary", size="lg")
                tasks_btn = gr.Button("Organize Tasks", variant="secondary", size="lg")
                translate_btn = gr.Button("Translate", variant="secondary", size="lg")

            # Dynamic options with better styling
            with gr.Group():
                question_input = gr.Textbox(
                    label="Your Question",
                    placeholder="What would you like to know?",
                    visible=False,
                    lines=2
                )

                study_duration = gr.Dropdown(
                    choices=["1 week", "2 weeks", "1 month", "3 months", "6 months"],
                    label="Study Duration",
                    value="1 month",
                    visible=False
                )

                learning_style = gr.Dropdown(
                    choices=["Visual Learner", "Step-by-step", "Quick Overview", "Detailed Explanation", "Interactive", "Problem-solving"],
                    label="Learning Style",
                    value="Step-by-step",
                    visible=False
                )

                language_select = gr.Dropdown(
                    choices=["Spanish", "French", "German", "Hindi", "Tamil", "Chinese", "Japanese", "Portuguese", "Italian", "Russian", "Arabic"],
                    label="Target Language",
                    value="Spanish",
                    visible=False
                )

            # Results area with enhanced styling
            gr.Markdown("## Results")
            output_area = gr.Textbox(
                label="AI Response",
                lines=15,
                max_lines=30,
                show_copy_button=True,
                placeholder="Your results will appear here...",
                elem_classes=["results-container"]
            )

        # Hidden state management
        current_mode = gr.State("explain")

        # Setup button functionality
        setup_btn.click(
            initialize_system,
            inputs=[api_key],
            outputs=[setup_status, main_interface]
        )

        # Dynamic content preview updates
        for input_component in [file_input, url_input, text_input]:
            input_component.change(
                lambda f, u, t: f'<div class="content-preview">{get_content_preview(f, u, t)}</div>',
                inputs=[file_input, url_input, text_input],
                outputs=[content_preview]
            )

        # Enhanced mode switching functions
        def show_question_mode():
            return (
                gr.update(visible=True), gr.update(visible=False),
                gr.update(visible=False), gr.update(visible=False),
                "questions"
            )

        def show_study_mode():
            return (
                gr.update(visible=False), gr.update(visible=True),
                gr.update(visible=False), gr.update(visible=False),
                "study_plan"
            )

        def show_tutor_mode():
            return (
                gr.update(visible=False), gr.update(visible=False),
                gr.update(visible=True), gr.update(visible=False),
                "tutor"
            )

        def show_translate_mode():
            return (
                gr.update(visible=False), gr.update(visible=False),
                gr.update(visible=False), gr.update(visible=True),
                "translate"
            )

        def hide_all_options(mode):
            return (
                gr.update(visible=False), gr.update(visible=False),
                gr.update(visible=False), gr.update(visible=False),
                mode
            )

        # Button click events
        questions_btn.click(
            show_question_mode,
            outputs=[question_input, study_duration, learning_style, language_select, current_mode]
        )

        study_btn.click(
            show_study_mode,
            outputs=[question_input, study_duration, learning_style, language_select, current_mode]
        )

        tutor_btn.click(
            show_tutor_mode,
            outputs=[question_input, study_duration, learning_style, language_select, current_mode]
        )

        translate_btn.click(
            show_translate_mode,
            outputs=[question_input, study_duration, learning_style, language_select, current_mode]
        )

        for btn, mode in [
            (explain_btn, "explain"), (flowchart_btn, "flowchart"),
            (summarize_btn, "summarize"), (tasks_btn, "tasks")
        ]:
            btn.click(
                lambda m=mode: hide_all_options(m),
                outputs=[question_input, study_duration, learning_style, language_select, current_mode]
            )

        # Main processing with progress tracking
        all_inputs = [file_input, url_input, text_input, current_mode, question_input, study_duration, learning_style, language_select]

        for btn in [explain_btn, questions_btn, flowchart_btn, summarize_btn, study_btn, tutor_btn, tasks_btn, translate_btn]:
            btn.click(
                handle_request,
                inputs=all_inputs,
                outputs=[output_area, status_display],
                show_progress="full"
            )

        # Help section
        with gr.Accordion("Quick Help & Tips", open=False):
            gr.Markdown("""
            ### How to use:
            1. **Setup**: Enter your Gemini API key (one-time setup)
            2. **Add Content**: Upload file, paste URL, or type directly
            3. **Choose Action**: Click any button for desired function
            4. **Get Results**: Watch progress and receive AI response

            ### Pro Tips:
            - **File Support**: PDF, PowerPoint, Text files up to 50MB
            - **URL Support**: Articles, blogs, documentation pages
            - **Languages**: 11+ languages for translation
            - **Progress**: Real-time progress tracking for all operations
            - **Copy Results**: Use copy button to save responses

            ### Device Info:
            - **GPU Available**: Faster processing with CUDA support
            - **CPU Fallback**: Works on any system configuration

            **Get API Key**: Free at [Google AI Studio](https://aistudio.google.com/app/apikey)
            """)

    return app

# Launch the application
if __name__ == "__main__":
    app = create_dynamic_interface()
    app.launch(
        debug=True,
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        inbrowser=True
    )